In [ ]:
import requests, re, time
import numpy as np

import torch, torchvision
from torch import nn, optim
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
# For HTML parsing
from bs4 import BeautifulSoup as BS
# For making csv files to store prices of paintings
import pandas as pd
#Price help
from re import sub
from decimal import Decimal

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
resp = requests.get("https://www.artmajeur.com/en/artworks/painting?saleOptions=sale&page=1")
soup = BS(resp.content.decode())

In [ ]:
#Testing scraping images
imgs = []

for tag in soup.find_all('figure'):
      imgs.append(tag.img.get('src'))

print(len(imgs))
print(imgs[30])



35
https://www.artmajeur.com/medias/mini/e/l/elewarne/artwork/14402504_marilyn.jpg


In [ ]:
#When I used the site prices shown in CAD but seems that scraper recieves prices in USD
#Testing scraping prices
prices = []
for tag in soup.find_all('div', class_='text-strong text-right ml-2'):
      price = tag.span.text
      if price[0] is not 'O':
        price = Decimal(sub(r'[^\d.]', '', price))
      
      prices.append(price)

print(len(prices))
print(imgs[25],prices[25])

35
https://www.artmajeur.com/medias/mini/a/r/arnaud-feuga/artwork/14297813_putting-au-trou-n-11.jpg?v=1615813275 1509.94


In [ ]:
#Testing scraping artists
artists = []
for tag in soup.find_all('a', class_='h6 artist-title font-weight-normal pb-0 mb-1'):
  artists.append(tag.text)
print(len(artists))
print(imgs[10],artists[10])

35
https://www.artmajeur.com/medias/mini/d/o/dominique-laude/artwork/14487614_en-attendant-hst-50x50cm-last.jpg 
                Laude
            


In [ ]:
#Going on website for me shows dimensions in cm, but scraper seems to recieve it in inches
#Testing scraping dimensions
heights = []
widths = []
for tag in soup.find_all('span', class_ = 'artwork-size'):
  text = tag.text
  height,width = text.split("x") 
  width, _ = width.split(" ")
  heights.append(height)
  widths.append(width)

print(len(widths))
print(heights[30], widths[30], imgs[30])


35

16.5 11.7 https://www.artmajeur.com/medias/mini/r/i/riina-sirel/artwork/14486750_15a90e85-a898-44b1-9034-8032c2049efe.jpg


In [ ]:
#PARSING MULTIPLE PAGES
imgs = []
artists = []
widths = []
heights = []
prices = []
for page in range(1, 278):
  resp = requests.get(f'https://www.artmajeur.com/en/artworks/painting?saleOptions=sale&page={page}')
  soup = BS(resp.content.decode())

  #imgs
  for tag in soup.find_all('figure'):
      imgs.append(tag.img.get('src'))
  #prices
  for tag in soup.find_all('div', class_='text-strong text-right ml-2'):
      price = tag.span.text
      if price[0] is not 'O':
        price = Decimal(sub(r'[^\d.]', '', price))
      
      prices.append(price)
  #artists
  for tag in soup.find_all('a', class_='h6 artist-title font-weight-normal pb-0 mb-1'):
      artists.append(tag.text)
  #dims
  for tag in soup.find_all('span', class_ = 'artwork-size'):
      text = tag.text
      height,width = text.split("x") 
      width, _ = width.split(" ")
      heights.append(height)
      widths.append(width)


In [ ]:
len(artists)

9796

In [ ]:
for artist in artists:
  artist = artist.strip('\n')



                Tatiana Rezvaya
            


In [ ]:
df = pd.DataFrame()
df.insert(loc=0, column = 'height (in)', value = heights)
df.insert(loc=0, column = 'width (in)', value = widths)
df.insert(loc=0, column = 'artist', value = artists)
df.insert(loc=0, column = 'price (USD)', value = prices)
df.insert(loc=0, column = 'image', value = imgs)

In [ ]:
df

,image,price (USD),artist,width (in),height (in)
0,https://www.artmajeur.com/medias/mini/t/a/tati...,2548.35,\n Tatiana Rezvaya\n,31.5,\n23.6
1,https://www.artmajeur.com/medias/mini/m/a/mary...,2211.46,\n Ryse Kaïa\n,45.7,\n35
2,https://www.artmajeur.com/medias/mini/t/a/tati...,1698.9,\n Tatiana Rezvaya\n,23.6,\n23.6
3,https://www.artmajeur.com/medias/mini/m/a/madb...,979.19,\n Madeline Berger (MadB)\n ...,24,\n18.1
4,https://www.artmajeur.com/medias/mini/t/a/tati...,3680.96,\n Tatiana Rezvaya\n,39.4,\n29.5
...,...,...,...,...,...
9791,https://www.artmajeur.com/medias/mini/s/t/stan...,280.32,\n Stan Bigda\n,9.1,\n9.1
9792,https://www.artmajeur.com/medias/mini/a/r/arti...,614.3,\n Violetta Golovchenko\n ...,10.6,\n10.6
9793,https://www.artmajeur.com/medias/mini/m/u/muri...,1188.05,\n Murielle Bozzia\n,24,\n25.2
9794,https://www.artmajeur.com/medias/mini/e/r/eric...,647.47,\n Eric Doisy\n,23.6,\n31.5


In [ ]:
clean_df= df[df['price (USD)'] != 'On Request']

In [ ]:
clean_df = clean_df.replace('\\n',' ', regex=True) 
clean_df = clean_df.sort_values('artist')
clean_df.reset_index(drop=True, inplace=True)

In [ ]:
clean_df

,image,price (USD),artist,width (in),height (in)
0,https://www.artmajeur.com/medias/mini/q/u/quat...,308.38,405,19.7,25.6
1,https://www.artmajeur.com/medias/mini/q/u/quat...,308.38,405,19.7,25.6
2,https://www.artmajeur.com/medias/mini/q/u/quat...,308.38,405,19.7,25.6
3,https://www.artmajeur.com/medias/mini/a/-/a-mh...,272.75,609,11.8,15.8
4,https://www.artmajeur.com/medias/mini/a/-/a-mh...,272.75,609,15.8,11.8
...,...,...,...,...,...
9384,https://www.artmajeur.com/medias/mini/a/n/anar...,629.04,Янина Руденко,17.7,17.7
9385,https://www.artmajeur.com/medias/mini/a/n/anar...,1199.1,Янина Руденко,27.6,31.5
9386,https://www.artmajeur.com/medias/mini/f/o/fova...,9583,⭐️Fov⭐️,51.2,70.9
9387,https://www.artmajeur.com/medias/mini/f/o/fova...,9583,⭐️Fov⭐️,51.2,70.9


In [ ]:
clean_df.to_csv('/content/drive/My Drive/CPEN291 Project/Artmajeur_Data.csv')